## 남방톱날꽃게(청게)의 나이 예측 딥러닝 모델 구현 3

본 소스 파일에서는 이전의 모델 1과는 다르게 과적합 예방을 위해 K겹 교차 검증 방식(K-fold Cross Validation)을 사용하였다.  
이전 딥러닝 모델 2와는 달리 EarlyStopping() 함수는 사용하지 않았다. 정확한 비교를 위해 다른 부분은 같게 설정하였다.   
다중 선형 회귀 분석의 방법을 사용하여 정확도를 MAE(Mean Absolute Error)를 이용해 평가하였다.

### 1. 필요한 라이브러리 import하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager   # 한글 사용을 위해 import
import numpy as np
import tensorflow as tf

tensorflow 버전 확인하기

In [2]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.3.0


tensorboard import & version 확인하기

In [3]:
import tensorboard

tensorboard.__version__

'2.4.0'

### 2. 데이터셋 가져오기

상대경로를 사용하여 최종적으로 가공된 데이터셋을 가져온다.

In [4]:
filepath = "./3_Dataset.csv"

In [5]:
df = pd.read_csv(filepath)

In [6]:
df

,Length,Diameter,Height,Weight,Viscera Weight,Shell Weight,Age,Sex_F,Sex_I,Sex_M
0,1.4375,1.1750,0.4125,24.635715,5.584852,6.747181,9,1,0,0
1,0.8875,0.6500,0.2125,5.400580,1.374951,1.559222,6,0,0,1
2,1.0375,0.7750,0.2500,7.952035,1.601747,2.764076,6,0,1,0
3,1.1750,0.8875,0.2500,13.480187,2.282135,5.244657,10,1,0,0
4,0.8875,0.6625,0.2125,6.903103,1.488349,1.700970,6,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3847,1.4625,1.1375,0.3250,24.819987,5.854172,6.378637,8,1,0,0
3848,1.5500,1.2125,0.4375,34.458817,7.172423,9.780577,10,1,0,0
3849,0.6250,0.4625,0.1625,2.012815,0.524466,0.637864,5,0,1,0
3850,1.0625,0.7750,0.2625,10.347568,2.338834,2.976698,6,0,1,0


### 3. 딥러닝을 위해 독립변수(x)와 종속변수(y)로 데이터 분리하기

In [7]:
#종속변수 
df_y = df['Age'].values
#독립변수
df_x = df.drop(['Age'], axis=1).values

In [8]:
df_x

array([[1.4375, 1.175 , 0.4125, ..., 1.    , 0.    , 0.    ],
       [0.8875, 0.65  , 0.2125, ..., 0.    , 0.    , 1.    ],
       [1.0375, 0.775 , 0.25  , ..., 0.    , 1.    , 0.    ],
       ...,
       [0.625 , 0.4625, 0.1625, ..., 0.    , 1.    , 0.    ],
       [1.0625, 0.775 , 0.2625, ..., 0.    , 1.    , 0.    ],
       [0.7875, 0.6125, 0.2125, ..., 0.    , 1.    , 0.    ]])

In [9]:
df_y

array([9, 6, 6, ..., 5, 6, 8], dtype=int64)

### 4.  K-겹 교차 검증(K-fold Cross Validation)을 사용한 딥러닝

sklearn에서 제공하는 StratifiedKFold 함수를 사용해 K-겹 교차 검증을 한다.  
StratifiedKFold 함수의 옵션 중에서 n_splits 부분은 K-겹에서 K에 해당하는 값이다.  
본 소스 코드에서는 5로 설정하여 5개의 그룹으로 데이터셋을 나누어 보았다.

In [10]:
from sklearn.model_selection import StratifiedKFold

# seed 값 설정
seed = 2

# k 값 설정
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed) 

In [11]:
skf 

StratifiedKFold(n_splits=5, random_state=2, shuffle=True)

X(독립변수)와 Y(종속변수)의 개수 출력

In [12]:
print("-> X의 개수 : ", len(df_x))
print(df_x)

print("\n-> Y의 개수 : ", len(df_y))
print(df_y)

-> X의 개수 :  3852
[[1.4375 1.175  0.4125 ... 1.     0.     0.    ]
 [0.8875 0.65   0.2125 ... 0.     0.     1.    ]
 [1.0375 0.775  0.25   ... 0.     1.     0.    ]
 ...
 [0.625  0.4625 0.1625 ... 0.     1.     0.    ]
 [1.0625 0.775  0.2625 ... 0.     1.     0.    ]
 [0.7875 0.6125 0.2125 ... 0.     1.     0.    ]]

-> Y의 개수 :  3852
[9 6 6 ... 5 6 8]


학습 데이터 개수와 테스트 데이터 개수 출력 

In [13]:
print("전체 데이터 개수 : ", len(df_x))

# 전체 데이터개수를 k에 해당하는 5로 나누고 반올림
print("각 데이터셋의 개수 : ", round(len(df_x)/5) )

# 학습 데이터의 개수  
print("학습 데이터 개수 : ", round(len(df_x)/5 * 4) )

# 테스트 데이터의 개수  
print("테스트 데이터 개수 : ", (len(df_x)- round(len(df_x)/5 * 4)) )

전체 데이터 개수 :  3852
각 데이터셋의 개수 :  770
학습 데이터 개수 :  3082
테스트 데이터 개수 :  770


### 5. 딥러닝 모델 설계, 컴파일 및 실행하기

MAE를 한 번 시행할 때마다 저장하여 한꺼번에 보여줄 수 있게 mae 리스트를 생성한다.

In [14]:
mae = []
loss = []
stage = 0

아래의 for문은 n_fold만큼 반복된다.  
데이터셋을 skf.split(df_x, df_y)를 이용해 학습 데이터셋과 테스트 데이터셋으로 나눈 후, 각각 train과 test에 할당한다.  
n_fold 마다 셔플이 적용되어 학습 데이터셋과 테스트 데이터셋이 교차될 수 있다.

df_x, df_y는 각각 3852개의 데이터가 있는 상태이며 skf로 셋팅된 환경으로 train, test로 shuffle되어 split된다.

In [15]:
import warnings

warnings.filterwarnings(action='ignore')

In [16]:
for train, test in skf.split(df_x, df_y): # skf로 셋팅된 환경에서 정의한 n_fold만큼 반복
    print("="*14, " training 데이터 개수 : ", len(train), ", test 데이터 개수 : ", len(test), "="*14)
    stage += 1
    print("-> K = %d : %d 번째 그룹에 있는 test 데이터셋 사용 " %(n_fold, stage))
    
    # 딥러닝 구조를 결정(모델을 설정), n_fold 번 반복됨
    # 이전 딥러닝 모델1과 같은 구조로 설정하여 검증 방식에 따른 정확도 차이 비교를 하고자 한다. 
    model = Sequential()
    #1번째 층 : 입력 x는 9개, 출력은 12개, 활성화함수 relu 
    model.add(Dense(12, input_dim=9, activation='relu'))
    #2번째 층 : 입력 x는 12개, 출력은 9개, 활성화함수 relu 
    model.add(Dense(8, activation='relu'))
    #3번째 층 : 입력 x는 9개, 출력은 1개, 활성화함수 linear
    model.add(Dense(1, activation='linear'))
    
    # 모델 컴파일    
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    # 모델 실행
    # 학습 데이터셋을 통한 학습: verbose가 2이면 함축적인 정보만 출력, epochs와 batch_sizes는 이전 모델과 동일하다.
    model.fit(df_x[train], df_y[train], validation_data=(df_x[test],df_y[test]), epochs=50, batch_size=10, verbose=2)
   
    # 테스트 데이터셋을 통한 검증 
    # 테스트 데이터셋 loss, mae(mae로 선형회귀 모델의 정확도 평가)
    k_loss, k_mae = model.evaluate(x=df_x[train], y=df_y[train], verbose=2)    
    # 테스트 데이터셋의 loss
    print("-> Test dataset evaluation : Loss = {:.4f}".format(k_loss))   
    # 테스트 데이터셋의 MAE
    print("-> Test dataset evaluation : MAE = {:.4f}".format(k_mae))
    
    # save k_loss to loss list
    loss.append(k_loss)
    
    # save k_mae to mae list
    mae.append(k_mae)

==============  training 데이터 개수 :  3081 , test 데이터 개수 :  771 ==============
-> K = 5 : 1 번째 그룹에 있는 test 데이터셋 사용 
Epoch 1/50
309/309 - 0s - loss: 16.4913 - mae: 3.2607 - val_loss: 12.2444 - val_mae: 2.6952
Epoch 2/50
309/309 - 0s - loss: 9.3964 - mae: 2.2794 - val_loss: 7.9141 - val_mae: 1.9489
Epoch 3/50
309/309 - 0s - loss: 6.7449 - mae: 1.8937 - val_loss: 6.3919 - val_mae: 1.8838
Epoch 4/50
309/309 - 0s - loss: 5.9806 - mae: 1.7821 - val_loss: 5.9706 - val_mae: 1.7146
Epoch 5/50
309/309 - 0s - loss: 5.7239 - mae: 1.7396 - val_loss: 5.6273 - val_mae: 1.7005
Epoch 6/50
309/309 - 0s - loss: 5.5505 - mae: 1.7068 - val_loss: 5.9478 - val_mae: 1.6547
Epoch 7/50
309/309 - 0s - loss: 5.4660 - mae: 1.6909 - val_loss: 5.4739 - val_mae: 1.6554
Epoch 8/50
309/309 - 0s - loss: 5.5240 - mae: 1.6930 - val_loss: 5.5135 - val_mae: 1.6329
Epoch 9/50
309/309 - 0s - loss: 5.4789 - mae: 1.6947 - val_loss: 5.4526 - val_mae: 1.6337
Epoch 10/50
309/309 - 0s - loss: 5.4078 - mae: 1.6770 - val_loss: 5.5963 - 

Epoch 38/50
309/309 - 0s - loss: 5.0985 - mae: 1.5949 - val_loss: 5.1671 - val_mae: 1.6462
Epoch 39/50
309/309 - 0s - loss: 5.0472 - mae: 1.6009 - val_loss: 5.0811 - val_mae: 1.6039
Epoch 40/50
309/309 - 0s - loss: 5.0585 - mae: 1.5973 - val_loss: 5.2385 - val_mae: 1.5715
Epoch 41/50
309/309 - 0s - loss: 5.0173 - mae: 1.5891 - val_loss: 5.1889 - val_mae: 1.6818
Epoch 42/50
309/309 - 0s - loss: 5.1224 - mae: 1.6103 - val_loss: 5.2417 - val_mae: 1.7048
Epoch 43/50
309/309 - 0s - loss: 5.0181 - mae: 1.5911 - val_loss: 5.5800 - val_mae: 1.8110
Epoch 44/50
309/309 - 0s - loss: 5.0263 - mae: 1.5940 - val_loss: 5.3224 - val_mae: 1.5814
Epoch 45/50
309/309 - 0s - loss: 5.0404 - mae: 1.5954 - val_loss: 5.2834 - val_mae: 1.5755
Epoch 46/50
309/309 - 0s - loss: 5.0470 - mae: 1.5999 - val_loss: 5.0713 - val_mae: 1.6131
Epoch 47/50
309/309 - 0s - loss: 5.0265 - mae: 1.5908 - val_loss: 5.1586 - val_mae: 1.5690
Epoch 48/50
309/309 - 0s - loss: 5.0073 - mae: 1.5767 - val_loss: 5.5100 - val_mae: 1.8099

Epoch 23/50
309/309 - 0s - loss: 25.8234 - mae: 4.0948 - val_loss: 25.0184 - val_mae: 4.0046
Epoch 24/50
309/309 - 0s - loss: 23.9390 - mae: 3.8966 - val_loss: 23.1951 - val_mae: 3.8113
Epoch 25/50
309/309 - 0s - loss: 22.1962 - mae: 3.7051 - val_loss: 21.5128 - val_mae: 3.6205
Epoch 26/50
309/309 - 0s - loss: 20.5973 - mae: 3.5159 - val_loss: 19.9683 - val_mae: 3.4326
Epoch 27/50
309/309 - 0s - loss: 19.0935 - mae: 3.3224 - val_loss: 18.4371 - val_mae: 3.2318
Epoch 28/50
309/309 - 0s - loss: 17.5470 - mae: 3.1124 - val_loss: 17.0321 - val_mae: 3.0499
Epoch 29/50
309/309 - 0s - loss: 16.2110 - mae: 2.9415 - val_loss: 15.8170 - val_mae: 2.8979
Epoch 30/50
309/309 - 0s - loss: 15.0110 - mae: 2.7787 - val_loss: 14.6394 - val_mae: 2.7331
Epoch 31/50
309/309 - 0s - loss: 13.9241 - mae: 2.6188 - val_loss: 13.6256 - val_mae: 2.5746
Epoch 32/50
309/309 - 0s - loss: 12.9355 - mae: 2.4841 - val_loss: 12.6687 - val_mae: 2.4610
Epoch 33/50
309/309 - 0s - loss: 12.0471 - mae: 2.3761 - val_loss: 11.

### 6. 딥러닝 평가하기

#### Loss(mse)를 K에 해당하는 횟수만큼 출력

In [17]:
print("K 값 :  %.f " % n_fold)
print("Loss : ", loss) #loss 리스트 값 출력

K 값 :  5 
Loss :  [5.035762310028076, 4.994692325592041, 5.117936611175537, 6.360251426696777, 5.065881729125977]


#### MAE를 K에 해당하는 횟수만큼 출력

In [18]:
print("K 값 :  %.f " % n_fold)
print("MAE : ", mae) # mae 리스트 값 출력 

K 값 :  5 
MAE :  [1.5784493684768677, 1.6452404260635376, 1.63773512840271, 1.6457065343856812, 1.6577982902526855]


#### 테스트 데이터셋을 기반으로 K번 iteration이 적용된 MSE 및 MAE의 평균값 계산

In [19]:
# 리스트형을 넘파이에서 제공하는 배열로 변환 
mse_ave = np.array(loss)
print(type(mse_ave)) 
print("\n MSE : %.4f" %(mse_ave.mean()))  # mse 평균값 출력 

<class 'numpy.ndarray'>

 MSE : 5.3149


In [20]:
mae_ave = np.array(mae)
print(type(mae_ave)) 
print("\n MAE : %.4f" %(mae_ave.mean()))  # mae 평균값 출력 

<class 'numpy.ndarray'>

 MAE : 1.6330


MAE 값이 1.6330가 나왔으므로 꽃게 나이 예측의 평균적인 오차값은 1.6330개월이라는 것을 알 수 있다.

### 7. 새로운 데이터를 위의 딥러닝 모델을 사용하여 예측하기 

keras의 Sequential 모델에서 제공하는 predict() 함수를 사용  
아직 학습과 테스트에 사용하지 않은 데이터를 사용해 예측하였다.

In [21]:
# Length, Diameter, Height, Weight, Viscera Weight, Shell Weight, Sex_F, Sex_I, Sex_M
crab1 = np.array([[1.5305, 1.185, 0.4135, 24.7356155, 5.5858415, 6.749191, 1, 0, 0]])
crab2 = np.array([[0.8975,0.67,0.2305,5.50050975,1.37645075,1.5602225,0,0,1]])

test_crab1 = model.predict(crab1)
test_crab2 = model.predict(crab2)

print("crab1 나이 예측 : %.f" %test_crab1, "개월")
print("crab2 나이 예측 : %.f" %test_crab2, "개월")

crab1 나이 예측 : 11 개월
crab2 나이 예측 : 8 개월


실제 데이터를 바탕으로 만든 가상의 꽃게 데이터에서 딥러닝 모델이 꽤 정확하게 동작함을 확인할 수 있었다.  
crab1은 데이터 셋의 첫 번째 crab 데이터의 각 독립변수들을 조금씩 증가시켜 생후 11개월 정도의 crab을 만들었고, crab2는 데이터 셋의 두 번째 crab 데이터의 각 독립변수들을 조금씩 증가시켜 생후 8개월 정도의 crab을 만든 것이다.

### 8. 학습 모델 저장하기

In [22]:
model.save('CrabAge3.h5') 

### 9. 분석

이전의 모델들과 마찬가지로 꽃게 나이 예측의 평균적인 오차값이 1.6330개월인 것은 예측이 잘 되었다고 생각한다. 그러나 처음 만든 홀드 아웃 교차 검증을 사용한 딥러닝 모델 1(오차: 1.5869개월)에 비해서 오차값이 크고, K겹 교차 검증 방식 자체가 시간이 상당히 오래 걸리는 검증 방식이기 때문에 비효율적이라고 생각한다. 또한 똑같은 K겹 교차 검증 방식을 사용한 모델 2(오차: 1.6262개월)와 비교해 보았을 때도 오차가 더 크게 나타났다. 나는 EarlyStopping() 함수가 오차를 크게 만드는데 기여하였다고 생각했는데 그것이 아니었다. 확실히 mae 리스트의 4번째 값은 EarlyStopping()를 사용하지 않아서 줄어들었지만, 다른 원소들의 오차값이 증가하여 MAE의 평균이 상승하는 효과를 불러일으켰다. 이를 통해 EarlyStopping() 함수는 오차를 줄이는데 효과적임을 확인할 수 있었다. 

그렇다면, K겹 교차 검증 방식이 홀드 아웃 교차 검증보다 더 큰 오차를 보이는 이유는 무엇일까? 과적합이 일어나지 않을 정도로 큰 테스트 데이터셋을 확보할 수 있으면 홀드 아웃 교차 검증의 성능이 더 좋은 것일까? 나는 학습 데이터셋과 테스트 데이터셋이 충분히 크고 데이터가 편향없이 고르게 분포되었기 때문에 30%의 테스트 데이터 셋을 확보한 홀드 아웃 교차 검증이 20%의 테스트 데이터 셋을 사용한 K겹 교차 검증 방식(이때 K=5)보다 우수한 성능을 보이는 일이 발생하였다고 생각한다. 여기에서 나는 K겹 교차 검증 방식의 개선 방안으로 iteration 마다 나오는 성능 평가 지표의 평균을 구하는 것이 아니라 중앙값을 구하는 방식을 생각하였다. 그렇게 된다면 본 모델과 이전의 딥러닝 모델 2의 경우에 K겹 교차 검증 방식의 오차는 더 줄어들게 된다. 특히 이전의 딥러닝 모델 2는 오차가 1.5885개월이 되어 딥러닝 모델1과 비슷한 정확도를 보여준다. 또는 데이터셋이 충분히 큰 위와 같은 경우에는 K = 3로 설정하여 3겹 교차 검증 방식을 사용한다면 33.3%가 테스트 데이터로 사용되기 때문에 더 좋은 성능을 보여줄 것이라 생각한다.